In [ ]:
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT license.

import helpermethods
import tensorflow as tf
import numpy as np
import sys
sys.path.insert(0, '../../')

#Provide the GPU number to be used
os.environ['CUDA_VISIBLE_DEVICES'] =''

#FastRNN and FastGRNN imports
from edgeml.trainer.fastTrainer import FastTrainer
from edgeml.graph.rnn import FastGRNNCell
from edgeml.graph.rnn import FastRNNCell

# Fixing seeds for reproducibility
tf.set_random_seed(42)
np.random.seed(42)

In [ ]:
#Loading and Pre-processing dataset for FastCells
#Ensure your usps10 went through dataCleanup.py before testing this out.
dataDir = "usps10/"
(dataDimension, numClasses, Xtrain, Ytrain, Xtest, Ytest) = helpermethods.preProcessData(dataDir)

In [ ]:
#Setting hyper-parameters for FastCells on USPS10 dataset

# Choose between FastGRNn & FastRNN
cell = "FastGRNN"

inputDims = 16 #features taken in by RNN in one timestep
hiddenDims = 32 #hidden state of RNN

totalEpochs = 300
batchSize = 100

learningRate = 0.01
decayStep = 200
decayRate = 0.1

outFile = None #provide your file, if you need all the logging info in a file

#low-rank parameterisation for weight matrices. None => Full Rank
wRank = None 
uRank = None 

#Sparsity of the weight matrices. x => 100*x % are non-zeros
sW = 1.0 
sU = 1.0

#Non-linearities for the RNN architecture. Can choose from "tanh, sigmoid, relu, quantTanh, quantSigm"
update_non_linearity = "tanh"
gate_non_linearity = "sigmoid"

In [ ]:
assert dataDimension % inputDims == 0, "Infeasible per step input, Timesteps have to be integer"

In [ ]:
#Placeholders for Data feeding during training and infernece
X = tf.placeholder("float", [None, int(dataDimension / inputDims), inputDims])
Y = tf.placeholder("float", [None, numClasses])

In [ ]:
#Creating a directory for current model in the datadirectory using timestamp
currDir = helpermethods.createTimeStampDir(dataDir, cell)
helpermethods.dumpCommand(sys.argv, currDir)

In [ ]:
#Create appropriate RNN cell object based on choice
if cell == "FastGRNN":
    FastCell = FastGRNNCell(hiddenDims, gate_non_linearity=gate_non_linearity,
                            update_non_linearity=update_non_linearity,
                            wRank=wRank, uRank=uRank)
elif cell == "FastRNN":
    FastCell = FastRNNCell(hiddenDims, update_non_linearity=update_non_linearity,
                           wRank=wRank, uRank=uRank)
else:
    sys.exit('Exiting: No Such Cell as ' + cell)

In [ ]:
#FastCell trainer for FastCell objects
FastCellTrainer = FastTrainer(FastCell, X, Y, sW=sW, sU=sU, learningRate=learningRate, outFile=outFile)

In [ ]:
#Session declaration and variable initialization. Interactive Session doesn't clog the entire GPU.
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [ ]:
#Fast Cell Train routine with 3 phase training
FastCellTrainer.train(batchSize, totalEpochs, sess, Xtrain, Xtest,
                      Ytrain, Ytest, decayStep, decayRate, dataDir, currDir)

In [1]:
#If one uses piece-wise linear approximations for non-linearities liek quant Tanh for tanh and quantSigm for Sigmoid, 
#they can benefit greatly from pure integer arithmetic after model quantization during prediction

#Model quantization
model_dir = "" #you wil lsee model dir printed at the end of trianing

import quantizeFastModels
quantizeFastModels.quantizeFastModels(model_dir)